In [2]:
#!pip install yfinance
#!pip install vectorbt

In [3]:
import vectorbt as vbt
import pandas_ta as ta
import pandas as pd
import warnings
import numpy as np
from pylab import *
import math
import time
from binance.spot import Spot 
import pandas as pd
import datetime

warnings.simplefilter(action='ignore',category=FutureWarning)
pd.set_option("display.max_rows",1000)

In [15]:
def check_value(Spot_client):
    #參數
    m15=15*60*1000 #15分鐘轉換成毫秒
    atr_times=1.5
    length=500
    data_length=500
    h=8
    mult=2.5
    k=2

    #抓資料
    start = time.time()*1000-m15*502
    end = time.time()*1000
    new_start=int(start)
    new_end=int(end)
    df=pd.DataFrame(Spot_client.klines("BTCUSDT", "15m",limit=1000,startTime=str(new_start),endTime=str(new_end)))
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    df=df[['datetime', 'open', 'high', 'low', 'close']].astype(float)
    df.index = [datetime.datetime.fromtimestamp(x/1000.0) for x in df.datetime]
    df=df.astype(float)

    #算ATR
    atr=ta.atr(df['high'],df['low'],df['close'],length=14)
    df['high_atr']=df['high']+atr*atr_times
    df['low_atr']=df['low']-atr*atr_times

    #計算500根nadaraya
    data=df[['open','high','low','close','high_atr','low_atr']][-500:]
    y = []
    sum_e = 0.
    for i in range(length):
        sum = 0.
        sumw = 0.
        for j in range(length):
            w = math.exp(-(math.pow(i-j,2)/(h*h*2)))
            sum += data['close'].iloc[j]*w
            sumw += w
        y2 = sum/sumw
        sum_e += abs(data['close'].iloc[i] - y2)
        y.append(y2)
    mae = sum_e/length*mult

    second_last_up_y2_mae=y[-2]+mae
    second_last_dn_y2_mae=y[-2]-mae
    second_last_high_atr=df['high_atr'][-2]
    second_last_low_atr=df['low_atr'][-2]
    second_last_close=df['close'][-2]

    print("\n前一根黃色上軌:",second_last_up_y2_mae,
        "\n前一根黃色下軌:",second_last_dn_y2_mae,
        "\n前一根綠色上ATR:",second_last_high_atr,
        "\n前一根紅色下ATR:",second_last_low_atr,
        "\n前一根收盤:",second_last_close)
        
    return second_last_up_y2_mae, second_last_dn_y2_mae, second_last_high_atr, second_last_low_atr,second_last_close 

In [19]:
def check_position(bin_client,now_state):
    bin_client.futures_account()['positions']

In [21]:
from binance.client import Client

api_key = 'rKkKfPQA9iQOwk7rGOPI2VmmXp6YxmZo6BAObtxpBt8OgxNaNz1eaO0NC9OAgKrY'
api_secret = '7srcTZ1sWi0gDUkV4Qv69fxLXTJ7ghw3zfEyISEFda1QJT9AATwfyfhHQDxp8NCR'

bin_client = Client(api_key, api_secret)


now_state=[
0,#0倉位有沒有做多了 
0,#1倉位有沒有做空了
0,#2綠ATR在上黃上，(等待收盤價破上黃線or平倉)
0,#3紅ATR在下黃下，(等待收盤價破下黃線or平倉)
0,#4還有沒有倉位
]

now_state=check_position(bin_client,now_state)
now_state

ModuleNotFoundError: No module named 'binance.client'

In [ ]:
Spot_client = Spot()

now_state=[
0,#0倉位有沒有做多了 
0,#1倉位有沒有做空了
0,#2綠ATR在上黃上，(等待收盤價破上黃線or平倉)
0,#3紅ATR在下黃下，(等待收盤價破下黃線or平倉)
0,#4還有沒有倉位
]

while 1:
    #確認數值
    second_last_up_y2_mae, second_last_dn_y2_mae, second_last_high_atr, second_last_low_atr,second_last_close =check_value(Spot_client)
    now_state=check_position(client,now_state)
    """
    #確認有沒有倉位被平倉
    if 都沒有倉位:
        多倉狀態=0
        空倉狀態=0
    elif 只有多倉:
        空倉狀態=0
    elif 只有空倉:
        多倉狀態=0

    #判斷多單現在是怎樣
    if 紅色在黃色之下，且沒有多倉:
        if 收盤價收在黃色下:
            做多!!
            多倉狀態=1

    if 現在有多倉:
        if 紅色線在黃色線之上:
            平倉了!
            多倉狀態=0

    #判斷空單現在是怎樣
    if 綠色在黃色之上，且沒有空倉:
        if 收盤價收在黃色上:
            做空!!
            空倉狀態=1

    if 現在有空倉:
        if 綠色線在黃色線之下:
            平倉了!
            空倉狀態=0

    str_txt="時間="+str(datetime.datetime.fromtimestamp(time.time()))+" ,有做多="+str(now_state[0])+" ,有做空="+str(now_state[1])+\
            " ,綠atr在黃上="+str(now_state[2])+" ,紅atr在黃下="+str(now_state[3])+" ,有個倉位="+str(now_state[4])+"\n"
    f = open("state.txt", "a")
    f.write(str_txt)
    f.close()
    """